In [1]:
! wget https://archive.ics.uci.edu/static/public/186/wine+quality.zip
! unzip wine+quality.zip -d wine_quality
! wget https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip
! unzip human+activity+recognition+using+smartphones.zip -d human_activity_recognition_using_smartphones

--2024-10-11 13:33:26--  https://archive.ics.uci.edu/static/public/186/wine+quality.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘wine+quality.zip’

wine+quality.zip        [ <=>                ]  89.21K  --.-KB/s    in 0.06s   

2024-10-11 13:33:26 (1.40 MB/s) - ‘wine+quality.zip’ saved [91353]

Archive:  wine+quality.zip
  inflating: wine_quality/winequality-red.csv  
  inflating: wine_quality/winequality-white.csv  
  inflating: wine_quality/winequality.names  
--2024-10-11 13:33:28--  https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: